## Exercise 55 - MOTIF

In [6]:
from graphframes import GraphFrame

inputPathVertexes = "/data/students/bigdata-01QYD/ex_data/Ex55/data/vertexes.csv"
inputPathEdges = "/data/students/bigdata-01QYD/ex_data/Ex55/data/edges.csv"
outputPath = "resOut_ex55/"

In [2]:
verticesDF = spark.read.load(inputPathVertexes, format="csv", header=True, inferSchema=True)
edgesDF = spark.read.load(inputPathEdges, format="csv", header=True, inferSchema=True)

In [3]:
verticesDF.show()

+---+----------+--------+
| id|entityName|    name|
+---+----------+--------+
| V1|      user|   Paolo|
| V2|     topic|     SQL|
| V3|      user|   David|
| V4|     topic|Big Data|
| V5|      user|    John|
+---+----------+--------+



In [4]:
edgesDF.show()

+---+---+----------+
|src|dst|  linktype|
+---+---+----------+
| V1| V2|      like|
| V1| V3|    follow|
| V1| V4|    follow|
| V3| V2|    follow|
| V3| V4|    follow|
| V5| V2|  expertOf|
| V2| V4|correlated|
| V4| V2|correlated|
+---+---+----------+



Output:
* The followed topics for each user
* One pair (user name, followed topic) per line
* Format: username, followed topic
* Use the CSV format to store the result

In [11]:
filteredEdges = edgesDF.filter("linktype='follow' ")
g = GraphFrame(verticesDF, filteredEdges)

selectedPaths = g.find("(x)-[follow]->(y)")
selectedPaths.show()

+-----------------+----------------+--------------------+
|                x|          follow|                   y|
+-----------------+----------------+--------------------+
|[V1, user, Paolo]|[V1, V3, follow]|   [V3, user, David]|
|[V1, user, Paolo]|[V1, V4, follow]|[V4, topic, Big D...|
|[V3, user, David]|[V3, V2, follow]|    [V2, topic, SQL]|
|[V3, user, David]|[V3, V4, follow]|[V4, topic, Big D...|
+-----------------+----------------+--------------------+



In [14]:
filteredSelectedPaths = selectedPaths.filter("y.entityName == 'topic' ")
filteredSelectedPaths.show()

+-----------------+----------------+--------------------+
|                x|          follow|                   y|
+-----------------+----------------+--------------------+
|[V1, user, Paolo]|[V1, V4, follow]|[V4, topic, Big D...|
|[V3, user, David]|[V3, V2, follow]|    [V2, topic, SQL]|
|[V3, user, David]|[V3, V4, follow]|[V4, topic, Big D...|
+-----------------+----------------+--------------------+



In [15]:
cleanedOutputDF = filteredSelectedPaths.selectExpr("x.name as username", "y.name as topic")
cleanedOutputDF.show()

+--------+--------+
|username|   topic|
+--------+--------+
|   Paolo|Big Data|
|   David|     SQL|
|   David|Big Data|
+--------+--------+



In [ ]:
cleanedOutputDF.write.csv(ouputPath, header=True)